In [2]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import plotly.express as px
import geopandas as gpd
from ipywidgets import interact, Dropdown, IntSlider

df = pd.read_excel('/Users/prithvi/Downloads/Final DataSheet.xlsx') #Read excel file

#To check for NA and 0 values    
na_values = df.isna().sum()

# Check for zero values
zero_values = df.eq(0).sum()

print("Number of NA values in each column:")
print(na_values)

print("\nNumber of zero values in each column:")
print(zero_values)

#Replace values
df.iloc[:, 2:] = df.iloc[:, 2:].replace(0, np.nan)

# Calculate the column-wise mean excluding NaN values
column_means = df.iloc[:, 2:].mean(skipna=True)

# Replace NaN values in each column with the column mean
for col in df.iloc[:, 2:].columns:
    df[col].fillna(column_means[col], inplace=True)

#Check 0 and NA values again
na_values = df.isna().sum()

# Check for zero values
zero_values = df.eq(0).sum()

print("Number of NA values in each column:")
print(na_values)

print("\nNumber of zero values in each column:")
print(zero_values)


df.to_excel('/Users/prithvi/Downloads/Processed Expat.xlsx', index=False) # used to make a dataframe into an excel sheet


Number of NA values in each column:
Sr No                               0
Country Name                        0
GDP                                 0
Death Rate                          0
Health Expenditure                  0
Life Expectancy                     0
Population Density                  0
Average Temperatures                3
Crime Rate                          2
Environmental Performance Index    12
Happiness Index                    37
CPI                                 0
Politoical Stability                0
dtype: int64

Number of zero values in each column:
Sr No                               0
Country Name                        0
GDP                                 9
Death Rate                          2
Health Expenditure                  9
Life Expectancy                     4
Population Density                  0
Average Temperatures                0
Crime Rate                          1
Environmental Performance Index     0
Happiness Index                     0


In [3]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

country_data_excel = pd.read_excel('/Users/prithvi/Documents/OIM_7502_classwork/Expat Final Project/Processed Expat.xlsx')# Path to the excel file
# Create dropdown widgets for the country selection
dropdown_country1 = widgets.Dropdown(
    options=country_data_excel['Country Name'].unique().tolist(),
    value='Afghanistan',
    description='Country 1:',
    disabled=False,
)

dropdown_country2 = widgets.Dropdown(
    options=country_data_excel['Country Name'].unique().tolist(),
    value='Albania',
    description='Country 2:',
    disabled=False,
)



%matplotlib inline

def update_plot(country1, country2):
    data_country1 = country_data_excel[country_data_excel['Country Name'] == country1]
    data_country2 = country_data_excel[country_data_excel['Country Name'] == country2]
    
    metrics = [ 'Life Expectancy', 'Crime Rate', 'Happiness Index', 'Crime Rate', 'Death Rate', 'Average Temperatures']
    values1 = data_country1[metrics].values.flatten()
    values2 = data_country2[metrics].values.flatten()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(metrics))
    
    ax.bar(index - bar_width/2, values1, bar_width, label=country1)
    ax.bar(index + bar_width/2, values2, bar_width, label=country2)
    
    ax.set_xlabel('Metrics')
    ax.set_ylabel('Values')
    ax.set_title('Comparison of Two Selected Countries')
    ax.set_xticks(index)
    ax.set_xticklabels(metrics)
    ax.legend()
    
    plt.show()
    
    
    # Link widgets to the update plot function
widgets.interactive(update_plot, country1=dropdown_country1, country2=dropdown_country2)

interactive(children=(Dropdown(description='Country 1:', options=('Afghanistan', 'Albania', 'Algeria', 'Andorr…

In [4]:

from ipywidgets import interact, Dropdown, IntSlider

# Assuming df is already loaded and contains your data
columns = [col for col in df.columns if col not in ['Sr No', 'Country Name']]

# Define a function to create the bubble plot with Plotly
def bubble_plot(x_variable, y_variable, top_n):
    # Sort the DataFrame by x_variable in descending order and select the top N rows
    top_countries = df.nlargest(top_n, x_variable)

    # Create a bubble chart using Plotly Express
    fig = px.scatter(top_countries, x=x_variable, y=y_variable,
                     size=x_variable,  # Customize the size by x_variable or another column
                     color='Country Name',  # Color by country
                     hover_name='Country Name',  # Show country name on hover
                     hover_data={x_variable: True, y_variable: True},  # Custom additional hover info
                     size_max=60)  # Max size of the bubble

    fig.update_layout(title=f'Top {top_n} Countries by {x_variable} vs {y_variable}',
                      xaxis_title=x_variable,
                      yaxis_title=y_variable,
                      title_x=0.5)  # Center the title

    fig.show()

# Define the dropdown widgets for selecting variables
x_dropdown = Dropdown(options=columns, description='X-axis Variable:')
y_dropdown = Dropdown(options=columns, description='Y-axis Variable:')

# Define the slider widget for selecting top N countries
top_n_slider = IntSlider(min=1, max=len(df), step=1, value=10, description='Top N Countries:')

# Define the interaction between widgets and the plot function
interact(bubble_plot, x_variable=x_dropdown, y_variable=y_dropdown, top_n=top_n_slider)


interactive(children=(Dropdown(description='X-axis Variable:', options=('GDP', 'Death Rate', 'Health Expenditu…

<function __main__.bubble_plot(x_variable, y_variable, top_n)>

In [5]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display

# Define the file paths
shapefile_path = gpd.datasets.get_path('naturalearth_lowres')
excel_file_path = '/Users/prithvi/Documents/OIM_7502_classwork/Expat Final Project/Processed Expat.xlsx'

# Read world shapefile
world = gpd.read_file(shapefile_path)

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

# Get the list of column headers (excluding 'Sr No' and 'Country Name')
variables = [col for col in df.columns if col not in ['Sr No', 'Country Name']]

# Function to update the map based on the selected variable
def update_map(selected_variable):
    # Merge the world shapefile with the selected variable from the DataFrame
    merged_data = world.merge(df[['Country Name', selected_variable]], how='left', left_on='name', right_on='Country Name')

    # Create the choropleth map using Plotly Express
    fig = px.choropleth(merged_data, 
                        locations='iso_a3', 
                        color=selected_variable,
                        hover_name='name',
                        projection='natural earth')

    # Update layout
    fig.update_layout(title=f'{selected_variable} Across Countries',
                      margin=dict(l=0, r=0, t=50, b=0))

    # Show the interactive plot
    fig.show()

# Create a dropdown widget for variable selection
dropdown = widgets.Dropdown(
    options=variables,
    value=variables[0],  # default value
    description='Metric:',
    disabled=False,
)

# Link dropdown to update function
widgets.interactive(update_map, selected_variable=dropdown)


/var/folders/2s/j_dmwqrs2g30ly77st875x140000gn/T/ipykernel_99403/4215194406.py:8: FutureWarning:

The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.



interactive(children=(Dropdown(description='Metric:', options=('GDP', 'Death Rate', 'Health Expenditure', 'Lif…

In [6]:
import gradio as gr
import pandas as pd

def load_data():
    
    excel_file_path = '/Users/prithvi/Documents/OIM_7502_classwork/Expat Final Project/Processed Expat.xlsx'  # Path to excel cocument
    try:
        data = pd.read_excel(excel_file_path)
        data.fillna(0, inplace=True)
        return data
    except Exception as e:
        raise RuntimeError(f"Failed to load data: {e}")

def get_parameters():
    """ Extract parameters from the DataFrame and format them for display. """
    data = load_data()
    parameters = data.columns.tolist()[2:]  # Adjust if columns change
    parameters_display = "\n".join([f"{i+1}. {param}" for i, param in enumerate(parameters)])
    return parameters_display

def evaluate_priorities(priority_indices, ranks):
    data = load_data()
    parameters = data.columns.tolist()[2:]  # Adjust if columns change

    # Parse priority indices and rank inputs
    priority_indices_list = list(map(int, priority_indices.split(',')))
    ranks_list = list(map(int, ranks.split(',')))

    top_5_priorities = [parameters[idx - 1] for idx in priority_indices_list]
    priority_ranking = dict(zip(top_5_priorities, ranks_list))

    total_rank = sum(range(1, 6))
    weights = {param: 0.23 - rank / total_rank * 0.05 for param, rank in priority_ranking.items()}

    # Calculate scores
    scores = {}
    for _, row in data.iterrows():
        country_score = sum(row[param] * weights[param] for param in top_5_priorities)
        scores[row['Country Name']] = country_score

    # Sort and return top 10 countries
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    top_10 = "\n".join(f"{i + 1}. {country} (Score: {score:.2f})" for i, (country, score) in enumerate(sorted_scores[:10]))
    return top_10

#  Gradio interface
iface = gr.Interface(
    fn=evaluate_priorities,
    inputs=[
        gr.Textbox(label="Enter the indices of your top 5 priorities (comma-separated, e.g., 1,2,3,4,5)"),
        gr.Textbox(label="Enter the ranks for these priorities (comma-separated, e.g., 5,4,3,2,1)")
    ],
    outputs="text",
    title="Top Countries Selector",
    description="Refer to the parameters below before entering indices and ranks:\n" + get_parameters()
)

iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
df = pd.read_excel('/Users/prithvi/Documents/OIM_7502_classwork/Expat Final Project/Processed Expat.xlsx') # Path to the excel file
def normalize_weights(weights):
    total = sum(weights)
    if total == 0:
        return [0] * len(weights)  # Avoid division by zero if all weights are zero
    return [float(w) / total for w in weights]

def compare_countries(country1, country2, gdp_weight, happiness_weight, life_expectancy_weight, crime_rate_weight, death_rate_weight, average_temperatures_weight):
    weights = [gdp_weight, happiness_weight, life_expectancy_weight, crime_rate_weight, death_rate_weight, average_temperatures_weight]
    normalized_weights = normalize_weights(weights)  # Normalize the weights

    # Update the dictionary with normalized weights
    weight_dict = {'GDP': normalized_weights[0], 'Happiness Index': normalized_weights[1], 'Life Expectancy': normalized_weights[2], 'Crime Rate' : normalized_weights[3], 'Death Rate': normalized_weights[4], 'Average Temperatures': normalized_weights[5]}

    score1 = calculate_country_score(country1, weight_dict)
    score2 = calculate_country_score(country2, weight_dict)
    
    if score1 > score2:
        return f"{country1} is recommended with a score of {score1:.2f} over {country2} with a score of {score2:.2f}."
    elif score2 > score1:
        return f"{country2} is recommended with a score of {score2:.2f} over {country1} with a score of {score1:.2f}."
    else:
        return f"Both {country1} and {country2} are equally preferable with identical scores of {score1:.2f}."

# Function to calculate the weighted score for a given country
def calculate_country_score(country, weights):
    country_data = df[df['Country Name'] == country]
    if country_data.empty:
        return 0  # Return 0 or handle this as needed if the country is not found
    score = (country_data['GDP'].values[0] * weights['GDP'] +
             country_data['Happiness Index'].values[0] * weights['Happiness Index'] +
             country_data['Life Expectancy'].values[0] * weights['Life Expectancy'] -
             country_data['Crime Rate'].values[0] * weights['Crime Rate']+
             country_data['Death Rate'].values[0] * weights['Death Rate']+
             country_data['Average Temperatures'].values[0] * weights['Average Temperatures']) 
    return score


iface = gr.Interface(
    fn=compare_countries,
    inputs=[
        gr.Dropdown(choices=list(df['Country Name'].unique()), label="First Country"),
        gr.Dropdown(choices=list(df['Country Name'].unique()), label="Second Country"),
        gr.Slider(minimum=0, maximum=1, value= 0.25, label="GDP Weight"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="Happiness Index Weight"),
        gr.Slider(minimum=0, maximum=1, value= 0.25, label="Life Expectancy Weight"),
        gr.Slider(minimum=0, maximum=1, value= 0.25, label="Crime Rate Weight"),
        gr.Slider(minimum=0, maximum=1, value= 0.25, label="Death Rate Weight"),
        gr.Slider(minimum=0, maximum=1, value= 0.25, label="Average Temperatures Weight"),
    ],
    outputs="text",
    title="Country Comparison for Retirement"
)

iface.launch()



Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
